analyse des sentiments 

In [2]:
# Installation de la librairie (Note le point d'exclamation au début)
!pip install textblob

# Téléchargement des dictionnaires de langue nécessaires
!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 2.5 MB/s eta 0:00:00
[nltk_data] Downloading package brown to
[nltk_data]     /Users/jessicabourdouxhe/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jessicabourdouxhe/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jessicabourdouxhe/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/jessicabourdouxhe/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/jessicabourdouxhe/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/jessicabourdouxhe/nltk_data...
^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  F

In [6]:
import pandas as pd
from textblob import TextBlob
import numpy as np

# 1. CHARGEMENT (On reprend ta base propre V8/Terminator)
# Assure-toi d'avoir le bon nom de fichier ici
try:
    df = pd.read_csv('/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/nlp_database_CLEAN_V8.csv')
    print("✅ Base V8 chargée.")
except:
    df = pd.read_csv('nlp_database_CLEAN_TERMINATOR.csv')
    print("✅ Base Terminator chargée.")

# 2. IDENTIFICATION DU TYPE (Discours vs Programme)
# On crée une colonne 'type' pour séparer les deux sources
def get_type(candidate_name):
    if "Party Platform" in str(candidate_name):
        return "Platform"
    else:
        return "Speech"

df['source_type'] = df['candidate'].apply(get_type)

print("Diagnostic des types :")
print(df['source_type'].value_counts())

# 3. CALCUL DES SCORES (TextBlob)
print("\n🧠 Calcul des scores en cours...")

def get_sentiment(text):
    return TextBlob(str(text)).sentiment.polarity

def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

df['sentiment'] = df['text'].apply(get_sentiment)
df['subjectivity'] = df['text'].apply(get_subjectivity)

# --- 4. LA MAGIE : PIVOT TABLE (Séparer les colonnes) ---

# On crée une table où 'Speech' et 'Platform' deviennent des COLONNES distinctes
df_pivot = df.pivot_table(
    index=['year', 'party'], 
    columns='source_type', 
    values=['sentiment', 'subjectivity'],
    aggfunc='mean'
).reset_index()

# On aplatit les noms de colonnes (ex: ('sentiment', 'Speech') -> 'sentiment_speech')
df_pivot.columns = ['year', 'party', 
                    'sentiment_platform', 'sentiment_speech', 
                    'subjectivity_platform', 'subjectivity_speech']

# --- 5. CALCUL DE LA MOYENNE (L'agrégat pour ton modèle) ---

# On calcule la moyenne des deux colonnes qu'on vient de créer
df_pivot['sentiment_mean'] = df_pivot[['sentiment_platform', 'sentiment_speech']].mean(axis=1)
df_pivot['subjectivity_mean'] = df_pivot[['subjectivity_platform', 'subjectivity_speech']].mean(axis=1)

# Tri par année
df_pivot = df_pivot.sort_values(by=['year', 'party'], ascending=[False, True])

# --- 6. APERÇU ET EXPORT ---

print("\n--- RÉSULTAT FINAL DÉTAILLÉ ---")
# On affiche un aperçu parlant (Année, Parti, Speech vs Platform)
print(df_pivot[['year', 'party', 'sentiment_speech', 'sentiment_platform', 'sentiment_mean']].head(10))

# Sauvegarde
output_name = 'political_sentiment_DETAILED.csv'
df_pivot.to_csv(output_name, index=False)
print(f"\n✅ Fichier '{output_name}' généré avec succès !")

✅ Base V8 chargée.
Diagnostic des types :
source_type
Speech      14
Platform    13
Name: count, dtype: int64

🧠 Calcul des scores en cours...

--- RÉSULTAT FINAL DÉTAILLÉ ---
    year       party  sentiment_speech  sentiment_platform  sentiment_mean
12  2024    Democrat          0.188678            0.107454        0.148066
13  2024  Republican          0.176667            0.089143        0.132905
10  2020    Democrat          0.154585            0.103591        0.129088
11  2020  Republican          0.157135                 NaN        0.157135
8   2016    Democrat          0.172119            0.131665        0.151892
9   2016  Republican          0.096610            0.097079        0.096845
6   2012    Democrat          0.157531            0.143446        0.150488
7   2012  Republican          0.193518            0.106685        0.150101
4   2008    Democrat          0.152744            0.123579        0.138161
5   2008  Republican          0.151983            0.122194        0.137089